In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import os
import codecs
import subprocess
from subprocess import Popen, PIPE, STDOUT

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [2]:
import numpy
import codecs
import tempfile
import cPickle
import os
import copy
from collections import OrderedDict

from fuel.datasets import H5PYDataset
from picklable_itertools import iter_, chain
from fuel.datasets import Dataset
from fuel.datasets import TextFile
from fuel.schemes import ConstantScheme
from fuel.streams import DataStream
from fuel.transformers import (
    Merge, Batch, Filter, Padding, SortMapping, Unpack, Mapping)

import os
import shutil
from collections import Counter
from theano import tensor
from toolz import merge
import numpy
import pickle
from subprocess import Popen, PIPE
import codecs

from blocks.algorithms import (GradientDescent, StepClipping,
                               CompositeRule, Adam, AdaDelta)
from blocks.extensions import FinishAfter, Printing, Timing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.filter import VariableFilter
from blocks.graph import ComputationGraph, apply_noise, apply_dropout
from blocks.initialization import IsotropicGaussian, Orthogonal, Constant
from blocks.main_loop import MainLoop
from blocks.model import Model
from blocks.select import Selector
from blocks.search import BeamSearch
from blocks_extras.extensions.plot import Plot

from machine_translation.checkpoint import CheckpointNMT, LoadNMT
from machine_translation.model import BidirectionalEncoder, Decoder
from machine_translation.sampling import BleuValidator, Sampler, SamplingBase
from machine_translation.stream import (get_tr_stream, get_dev_stream,
                                        _ensure_special_tokens)


from nnqe.dataset.preprocess import whitespace_tokenize

In [3]:
# make a fuel stream which subclasses text file to create a stream which provides three sources: 
# (source, [samples], and BLEU scores)

# going forward, this may not be the fastest way because the sampling and BLEU score computation can be time consuming
# we should look at Fuel's read-ahead and cacheing capacity

# TODO: stateful transformer which takes a stream and adds the sources ('samples', 'scores')
# class Mapping(Transformer)
# the mapping should be a callable which gets samples, then computes the sentence-level BLEU
# score for each sample with respect to the reference

# use this script to get sentence-level scores(?)
# https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/mteval-v13a.pl

# parameters
# sample_func: function(num_samples=1) which takes source seq and outputs <num_samples> samples
# score_func: function

# TODO: how do they create simple test streams in the fuel tests?

In [4]:


class MTSampleStreamTransformer(object):
    """
    Stateful transformer which takes a stream and adds the sources ('samples', 'scores')
    
    Parameters
    ----------
    sample_func: function(num_samples=1) which takes source seq and outputs <num_samples> samples
    score_func: function

    At call time, we expect a stream providing (sources,) -- i.e. something like a TextFile object

    
    """
    
    
    def __init__(self, sample_func, score_func):
        self.sample_func = sample_func
        self.score_func = score_func

    def __call__(self, source_data):
        return ([x for x in source_data[0]])





In [5]:
def fake_score(source, target):
    return 1.

def fake_sample(source):
    return [[2,67,33,778,323,68], [545,5347,432,21,53,68,47,3689,3]]

In [6]:
test_transformer = MTSampleStreamTransformer(fake_sample, fake_score)

In [7]:
fake_source = ([1,55,75,324,43,0],)

test_transformer(fake_source)

[1, 55, 75, 324, 43, 0]

In [17]:
# create the graph which can sample from our model 
# Note that we must sample instead of getting the 1-best, because we need the randomness to make the expected
# BLEU score make sense
# TODO: check on the parameterization of the softmax emitter sampler in blocks -- is softmax temperature implemented?

exp_config = {
    'src_vocab_size': 20000,
    'trg_vocab_size': 20000,
    'enc_embed': 300,
    'dec_embed': 300,
    'enc_nhids': 800,
    'dec_nhids': 800,
    'saved_parameters': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
    '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz',
    'src_vocab': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0'+\
    '.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/vocab.en-de.en.pkl',
    'trg_vocab': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0'+\
    '.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/vocab.en-de.de.pkl',
    'source_file': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
    '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/training_data/train.en.tok.shuf',
    'unk_id':1
}

def get_sampling_model_and_input(exp_config):
    # Create Theano variables
    encoder = BidirectionalEncoder(
        exp_config['src_vocab_size'], exp_config['enc_embed'], exp_config['enc_nhids'])

    decoder = Decoder(
        exp_config['trg_vocab_size'], exp_config['dec_embed'], exp_config['dec_nhids'],
        exp_config['enc_nhids'] * 2)

    # Create Theano variables
    logger.info('Creating theano variables')
    sampling_input = tensor.lmatrix('source')

    # Get beam search
    logger.info("Building sampling model")
    sampling_representation = encoder.apply(
        sampling_input, tensor.ones(sampling_input.shape))
    generated = decoder.generate(sampling_input, sampling_representation)

#     _, samples = VariableFilter(
#         bricks=[decoder.sequence_generator], name="outputs")(
#                  ComputationGraph(generated[1]))  # generated[1] is next_outputs
#     beam_search = BeamSearch(samples=samples)

    # build the model that will let us get a theano function from the sampling graph
    logger.info("Creating Sampling Model...")
    sampling_model = Model(generated)

    # Set the parameters from a trained models
    logger.info("Loading parameters from model: {}".format(exp_config['saved_parameters']))
    # load the parameter values from an .npz file
    param_values = LoadNMT.load_parameter_values(exp_config['saved_parameters'])
    LoadNMT.set_model_parameters(sampling_model, param_values)
    
    return sampling_model, sampling_input

test_model, theano_sampling_input = get_sampling_model_and_input(exp_config)

INFO:root:Creating theano variables
INFO:root:Building sampling model
INFO:root:Creating Sampling Model...
INFO:root:Loading parameters from model: /home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz
INFO:machine_translation.checkpoint: Loaded to CG (800,)         : /bidirectionalencoder/bidirectionalwmt15/backward.initial_state
INFO:machine_translation.checkpoint: Loaded to CG (1600,)        : /bidirectionalencoder/back_fork/fork_gate_inputs.b
INFO:machine_translation.checkpoint: Loaded to CG (300, 1600)    : /bidirectionalencoder/back_fork/fork_gate_inputs.W
INFO:machine_translation.checkpoint: Loaded to CG (800,)         : /bidirectionalencoder/back_fork/fork_inputs.b
INFO:machine_translation.checkpoint: Loaded to CG (300, 800)     : /bidirectionalencoder/back_fork/fork_inputs.W
INFO:machine_translation.checkpoint: Loaded to C

In [18]:
# make TextFile stream for source data
# Load dictionaries and ensure special tokens exist

def get_source_stream(source_file=None, src_vocab=None, src_vocab_size=30000,
                      unk_id=1, **kwargs):
    """Get a stream for source data"""
    source_stream = None
   
    src_vocab = _ensure_special_tokens(
        src_vocab if isinstance(src_vocab, dict) else
        cPickle.load(open(src_vocab)),
        bos_idx=0, eos_idx=src_vocab_size - 1, unk_idx=unk_id)
    source_dataset = TextFile([source_file], src_vocab, bos_token=None)
    source_stream = DataStream(source_dataset)
    return source_stream

In [19]:
test_source_stream = get_source_stream(**exp_config)

In [20]:
test_source_data = list(test_source_stream.get_epoch_iterator())

In [21]:
len(test_source_data)
a_few_instances = [numpy.array(i[0]) for i in test_source_data[:5]]

In [22]:
# test that we can pull samples from the model
test_sampling_func = test_model.get_theano_function()


# def _get_true_length(self, seq, vocab):
#     try:
#         return seq.tolist().index(vocab['</S>']) + 1
#     except ValueError:
#         return len(seq)

In [28]:
# load src and target vocabs to validate samples
src_vocab = cPickle.load(open(exp_config['src_vocab']))
trg_vocab = cPickle.load(open(exp_config['trg_vocab']))
src_vocab_size = exp_config['src_vocab_size'] - 1
trg_vocab_size = exp_config['trg_vocab_size'] - 1


src_vocab = _ensure_special_tokens(src_vocab, bos_idx=0,
                                   eos_idx=src_vocab_size, unk_idx=exp_config['unk_id'])
trg_vocab = _ensure_special_tokens(trg_vocab, bos_idx=0,
                                   eos_idx=trg_vocab_size, unk_idx=exp_config['unk_id'])

src_ivocab = {v:k for k,v in src_vocab.items()}
trg_ivocab = {v:k for k,v in trg_vocab.items()}



In [54]:
# inp = input_[i, :input_length]

def _get_true_length(seq, vocab):
    try:
        return seq.tolist().index(vocab['</S>']) + 1
    except ValueError:
        return len(seq)

# outputs of self.sampling_fn = outputs of sequence_generator.generate: next_states + [next_outputs] +
#                 list(next_glimpses.values()) + [next_costs])
num_samples = 5

for _ in range(num_samples):
    for source_seq in a_few_instances[:5]:
        print(' '.join(src_ivocab[idx] for idx in source_seq))
        _1, outputs, _2, _3, costs = test_sampling_func(source_seq[None, :])
        # the output is a [seq_len, 1] array
        outputs = outputs.reshape(outputs.shape[0])
        outputs = outputs[:_get_true_length(outputs, trg_vocab)]
        print(' '.join(trg_ivocab[idx] for idx in outputs))

# TODO: can we get multiple outputs in one shot?


Two boys in blue shirts sit on the end of a bench next to a fair ride . </S>
Zwei Jungen in blauen Shirts sitzen am Ende einer Bank neben einem Fahrgeschäft in der Stadt . </S>
A man pointing to the audience area on stage while holding a microphone with banjos behind him . </S>
Ein Mann zeigt auf den Bereich ist auf die Bühne riss . </S>
A little girl riding a two wheeler , wearing her helmet close to the street with passing cars . </S>
Ein kleines Mädchen fährt auf einem zwei Fahrrädern , bei dem Verkehr auf der Straße in der Nähe ist viele Zeichen hindurch . </S>
A road worker in green , there is an orange cone near him . </S>
Eine Straße in grün , neben ihm befindet sich der Person . </S>
A man leans against a building near a busy street . </S>
Ein Mann lehnt an einem Gebäude in der Nähe einer stark befahrenen Straße . </S>
Two boys in blue shirts sit on the end of a bench next to a fair ride . </S>
Zwei Jungen in blauem Hemden sitzen am Ende einer kniet neben einem Fahrgeschäft in 

In [ ]:
    outputs = outputs.reshape

In [16]:
# calling the external scoring script
    
    self.multibleu_cmd = ['perl', self.config['bleu_script'],
                              self.config['val_set_grndtruth'], '<']
    
    mb_subprocess = Popen(self.multibleu_cmd, stdin=PIPE, stdout=PIPE)
    
      mb_subprocess.stdin.flush()

    
     # Write to subprocess and file if it exists
    print(trans_out, file=mb_subprocess.stdin)
            
            
        # send end of file, read output.
        mb_subprocess.stdin.close()
        stdout = mb_subprocess.stdout.readline()
        logger.info(stdout)
        out_parse = re.match(r'BLEU = [-.0-9]+', stdout)
        logger.info("Validation Took: {} minutes".format(
            float(time.time() - val_start_time) / 60.))
        assert out_parse is not None

        # extract the score
        bleu_score = float(out_parse.group()[6:])
        self.val_bleu_curve.append(bleu_score)
        logger.info(bleu_score)
        mb_subprocess.terminate()


        return bleu_score

IndentationError: unexpected indent (<ipython-input-16-f1dfc900db91>, line 3)

In [ ]:
# Model related -----------------------------------------------------------
# Sequences longer than this will be discarded
'seq_len': 40
# Number of hidden units in encoder/decoder GRU
'enc_nhids': &REC_SIZE 800
'dec_nhids': 800

# Dimension of the word embedding matrix in encoder/decoder
'enc_embed': &EMBED_SIZE 300
'dec_embed': 300

# Optimization related ----------------------------------------------------
# Batch size
'batch_size': &BATCH_SIZE 15 

# This many batches will be read ahead and sorted
'sort_k_batches': 15 

# Optimization step rule
'step_rule': &STEP_RULE 'Adam'

# Gradient clipping threshold
'step_clipping': 1.

# Std of weight initialization
'weight_scale': 0.01

# Regularization related --------------------------------------------------

# Weight noise flag for feed forward layers
'weight_noise_ff': &FF_NOISE False

# Weight noise flag for recurrent layers
'weight_noise_rec': False

# Dropout ratio, applied only after readout maxout
'dropout': &DROPOUT 0.3

# Source and target vocabulary sizes, should include bos, eos, unk tokens
'src_vocab_size': &SRC_VOCAB_SIZE 20000
'trg_vocab_size': &TGT_VOCAB_SIZE 20000

# Special tokens and indexes
'unk_id': 1
'bos_token': '<S>'
'eos_token': '</S>'
'unk_token': '<UNK>'

# Root directory for dataset
'datadir': &DATADIR /media/1tb_drive/multilingual-multimodal/flickr30k/train/processed

# the name of the directory where the model will be saved and checkpointed
#'model_save_directory': &SAVEDIR !format_str ['unbabel_data_dropout{}_ff_noise{}_search_model_en2es_vocab{}_emb{}_rec{}_batch{}', *DROPOUT, *FF_NOISE, *SRC_VOCAB_SIZE, *EMBED_SIZE, *REC_SIZE, *BATCH_SIZE]
'model_save_directory': &SAVEDIR !format_str ['BERTHA-TEST_{}_wmt-multimodal_internal_data_dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15', *STEP_RULE]

# Where to save model, this corresponds to 'prefix' in groundhog
'saveto': &OUTPUT_DIR !path_join [*DATADIR, *SAVEDIR]

# Module name of the stream that will be used
# note this requires the stream to be implemented as a module -- there may be a better way
'stream': 'stream'

# Source and target vocabularies
'src_vocab': !path_join [*DATADIR, 'vocab.en-de.en.pkl']
'trg_vocab': !path_join [*DATADIR, 'vocab.en-de.de.pkl']

# Source and target datasets
'src_data': !path_join [*DATADIR, 'train.en.tok.shuf']
'trg_data': !path_join [*DATADIR, 'train.de.tok.shuf']


# Early stopping based on BLEU score on dev set ------------------------------------

# Normalize cost according to sequence length after beam-search
'normalized_bleu': True

# Bleu script that will be used (moses multi-perl in this case)
'bleu_script': !path_join [*DATADIR, 'multi-bleu.perl']

# Validation set source file
'val_set': !path_join [*DATADIR, 'dev.en.tok']

# Validation set gold file
'val_set_grndtruth': !path_join [*DATADIR, 'dev.de.tok']

# Print validation output to file
'output_val_set': True

# Validation output file
'val_set_out': !path_join [*OUTPUT_DIR, 'validation_out.txt']

# Beam-size
'beam_size': 20 

# Timing/monitoring related -----------------------------------------------

# Maximum number of updates
'finish_after': 1000000

# Reload model from files if exist
'reload': True

# Save model after this many updates
'save_freq': 5000

# Show samples from model after this many updates
'sampling_freq': 5000

# Show this many samples at each sampling
'hook_samples': 5

# Validate bleu after this many updates
'bleu_val_freq': 1000 

# Start bleu validation after this many updates
'val_burn_in': 5000

# Using trained models for prediction ------------

# The location of the saved parameters of a trained model as .npz
'saved_parameters': ~

# The location of a test set in the source language
'test_set': ~


In [ ]:
class MTSampleStreamGenerator(Dataset):
    """

    Parameters
    ----------
    source_files: list[str] - the files containing the source seqs we'll use to generate samples
    sample_func: function(num_samples=1) which takes source seq and outputs <num_samples> samples
    score_func: function
    
    # source dict only or source + target dict?
    dictionary: dict mapping word-->id -- TODO: switch to parameterized choice

    """

    provides_sources = ('sources','targets', 'scores')
    example_iteration_scheme = None

    def __init__(self, files, dictionary, all_idxs_and_probs, bos_token=None, eos_token=None,
                 unk_token='<UNK>', level='word', preprocess=None):
        self.files = files
        self.dictionary = dictionary
        # use all idxs to choose random substitutions and insertions
        # WORKING: switch to parameterized choice using word freq
        # WORKING pass in two lists -- words and probs (use the p= kwarg from np.random.choice)
        # self.all_idxs = dictionary.values()
        self.all_idxs, self.all_probs = zip(*all_idxs_and_probs.items())
        if bos_token is not None and bos_token not in dictionary:
            raise ValueError
        self.bos_token = bos_token
        if eos_token is not None and eos_token not in dictionary:
            raise ValueError
        self.eos_token = eos_token
        if unk_token not in dictionary:
            raise ValueError
        self.unk_token = unk_token
        if level not in ('word', 'character'):
            raise ValueError
        self.level = level
        self.preprocess = preprocess
        super(RandomTargetGenerator, self).__init__()

    def open(self):
        return chain(*[iter_(open(f)) for f in self.files])

    def get_data(self, state=None, request=None):
        if request is not None:
            raise ValueError
        sentence = next(state)
        if self.preprocess is not None:
            sentence = self.preprocess(sentence)
        pe_sequence = [self.dictionary[self.bos_token]] if self.bos_token else []
        if self.level == 'word':
            pe_sequence.extend(self.dictionary.get(word,
                                            self.dictionary[self.unk_token])
                        for word in sentence.split())
        else:
            pe_sequence.extend(self.dictionary.get(char,
                                            self.dictionary[self.unk_token])
                        for char in sentence.strip())
        if self.eos_token:
            pe_sequence.append(self.dictionary[self.eos_token])

        # now use the pe sequence to generate artificial target_sequence and tag_sequence
        target_sequence, tag_sequence = self._generate_artificial_target_and_tag_seqs(pe_sequence)

        return (target_sequence, tag_sequence)

    # TODO: currently only supports subs, not inserts
    # TODO: tag identities are hard-coded as 0=BAD 1=OK
    def _generate_artificial_target_and_tag_seqs(self, pe_sequence):
        target_sequence = copy.copy(pe_sequence)
        tag_sequence = [1] * len(pe_sequence)

        # TODO: make num_subs distribution configurable
        num_subs_to_make = numpy.random.randint(0, (int(numpy.ceil(len(pe_sequence / 1.3)))))
        sub_idxs = numpy.random.choice(len(pe_sequence), size=num_subs_to_make)
#         print('len orig seq: {}'.format(len(pe_sequence)))
#         print('len orig seq / 2: {}'.format(len(pe_sequence) //2))
#         print('n_subs: {}'.format(num_subs_to_make))
#         print(sub_idxs)
        for idx in sub_idxs:
            new_word = numpy.random.choice(self.all_idxs, p=self.all_probs)
            target_sequence[idx] = new_word
            tag_sequence[idx] = 0

        return target_sequence, tag_sequence
